In [3]:
import xarray as xr
import rioxarray
import geopandas as gpd

# Step 1: Load NetCDF
ds = xr.open_dataset("chlorophyll-a.nc")  # adjust variable names if needed

# Select the variable of interest, for example:
da = ds['Water_body_chlorophyll_a']


ImportError: /opt/conda/lib/python3.10/site-packages/rasterio/../../../libgdal.so.36: undefined symbol: sqlite3_total_changes64

In [3]:
# Set spatial dims and CRS manually
da.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
da.rio.write_crs("EPSG:4326", inplace=True)  # Use correct EPSG here


<xarray.DataArray 'Water_body_chlorophyll_a' (time: 4, depth: 12, lat: 321,
                                              lon: 861)> Size: 53MB
[13266288 values with dtype=float32]
Coordinates:
  * lon          (lon) float64 7kB -6.0 -5.95 -5.9 -5.85 ... 36.9 36.95 37.0
  * lat          (lat) float64 3kB 30.0 30.05 30.1 30.15 ... 45.9 45.95 46.0
  * depth        (depth) float64 96B 0.0 5.0 10.0 20.0 ... 125.0 150.0 200.0
  * time         (time) datetime64[ns] 32B 2007-02-16 2007-05-16 ... 2007-11-16
    spatial_ref  int64 8B 0
Attributes:
    units:          mg m-3
    standard_name:  mass_concentration_of_chlorophyll_a_in_sea_water
    long_name:      Mass concentration of chlorophyll-a in sea water
    cell_methods:   time: mean within years time: mean over years

In [4]:
gdf = gpd.read_file("../shp/dissolto med.shp")
print(gdf.crs)


EPSG:4326


In [5]:
# Reproject shapefile to match NetCDF
gdf = gdf.to_crs(da.rio.crs)

# Clip
clipped = da.rio.clip(gdf.geometry, gdf.crs)


In [6]:
clipped.to_netcdf("clipped_output.nc")